In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics
else:
    os.chdir('..')

!pip install -r requirements.txt

!pip install -e .

In [2]:
#autoreload modules
%load_ext autoreload
%autoreload 2

from src.user import User

In [3]:
ibai = User.from_name("ibai")
ibai_follows = ibai.follows
print(f"{ibai.name.capitalize()} follows {len(ibai_follows)} users")

Ibai follows 155 users


In [4]:
users_that_ibai_follows = User.get_users(ibai_follows)
print("%s users retrieved from the list of follows"%len(users_that_ibai_follows))

155 users retrieved from the list of follows


In [5]:
import pandas as pd
df = pd.DataFrame([ibai]+users_that_ibai_follows)
df

,id,name,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286976290,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,35980866,Perxitaa,partner,Troll de Youtube http://www.youtube.com/user/p...,es,Grand Theft Auto V,38761384,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-05T22:12:47Z,None
2,145908612,momoladinastia,partner,"Siempre al pie para los mios, Stream #ATR re b...",es,Just Chatting,11215548,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-01-25T01:52:25Z,None
3,205218019,el_yuste,partner,Lo que viene a ser Yuste diciendo sus cosas.,es,Just Chatting,1095414,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-03-13T21:58:12Z,None
4,44880944,ElmiilloR,partner,CHALLENGER RANK 1 EN TODOS LOS GAMES JAMÁS CRA...,es,Slots,78622482,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-06-17T18:54:08Z,None
...,...,...,...,...,...,...,...,...,...,...
151,48589777,Ethieen,partner,Juego a cosas mientas hablo de la vida,es,Just Chatting,7357861,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-05T17:49:05Z,None
152,62951382,AKAWonder,partner,Mi nombre es Esteban Serrano aunque soy más co...,es,Teamfight Tactics,23452833,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-05-21T22:26:00Z,None
153,50244027,Kuentinlol,partner,,es,League of Legends,2955380,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-10-16T20:05:00Z,None
154,91816176,FTREFORMED,partner,"Antes caster en LVP, ahora head coach de 19esp...",es,Golden Sun: Dark Dawn,1995472,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-05-24T13:10:33Z,None


In [9]:
from src.data import make_data_from_root_user

# Obtiene datos de cada usuario del arbol empezando por Ibai y sus follows
# cada iteracion siguiente coge un usuario del arbol aleatorio para añadir sus follows 
# y asi hasta llegar a max_users o que se interrumpa manualmente. 
# Cada 50 iteraciones guarda el progreso actual en un archivo csv
make_data_from_root_user("esies5",output_filepath="data/data.csv",max_users=500)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: esies5
INFO:src.data.make_dataset:New users will be fetched until 500 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 6 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:max_users reached. 523 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...


In [10]:
data = pd.read_csv("data/data.csv")
#data.drop_duplicates("id", keep='first',inplace=True)
data.fillna({"broadcaster_type":"user"},inplace=True)